In [17]:
from qiskit import Aer, assemble, QuantumCircuit, QuantumRegister, ClassicalRegister
from qiskit.visualization import plot_histogram
from qiskit.quantum_info import Statevector
from numpy import sqrt, pi

In [20]:
def createInputState(qc, reg, n):
    qc.barrier()
    qc.h(reg[n])
    for i in range(n):
        qc.cp(pi/(2**(i+1)), reg[n-(i+1)], reg[n])

def evolveQFTstate(qc, reg_a, reg_b, n):
    for i in range(n+1):
        qc.cp(pi/(2**(i)), reg_b[n-i], reg_a[n])
        
def inverseQFT(qc, reg, n):
    for i in range(n):
        qc.cp(-pi/(2**(n-i)), reg[i], reg[n])
    qc.h(reg[n])

def add(first, second, n):
    a = QuantumRegister(n+1, "a")
    b = QuantumRegister(n+1, "b")
    cl = ClassicalRegister(n+1, "cl")
    qc = QuantumCircuit(a, b, cl, name="qc")
    for i in range(n):
        if first[i] == '1':
            qc.x(a[n-(i+1)])
    for i in range(n):
        if second[i] == '1':
            qc.x(b[n-(i+1)])
    
    for i in range(n+1):
        createInputState(qc, a, n-i)
    for i in range(n+1):
        evolveQFTstate(qc, a, b, n-i)
    for i in range(n+1):
        inverseQFT(qc, a, i)
    
    for i in range(n+1):
        qc.measure(a[i], cl[i])
        
    sim = Aer.get_backend('aer_simulator')
    result = sim.run(assemble(qc)).result().get_counts()
    return result

In [22]:
first = bin(int(input()))
second = bin(int(input()))

first, second = first[2:], second[2:]
first, second = first.zfill(9), second.zfill(9)

n = len(first)
result = add(first, second, n)
print(result)

4
5
{'0000001001': 1024}
